- 컨텍스트 매니저: 원하는 타이밍에 정확하게 리소스를 할당, 제공, 반환하는 역할

In [1]:
# Ex1

file = open('./testfile1.txt','w')

try:
    file.write('Context Manager Test1 \n Contextlib Test1')

finally:
    file.close()
    

In [2]:
# Ex2

with open('./testfile2.txt','w') as f:
    f.write('Context Manager Test1 \n Contextlib Test1')

위 Ex1과 Ex2는 동일한 기능을 수행한다.<br>
다만, Ex2의 경우 with문 안에 있는 작업이 끝나면 Ex1처럼 file.close()를 하지 않아도 자동으로 반환이 된다는 것<br>
with문 안에 file.close()가 파이썬 내부에 build되어서 작동하고 있음.<br>
우리가 with문을 customizing해서 file뿐만 아니라 인터넷 접속하고 사용한 후 나올때 연결 끊는 등 우리만의 with문 사용할 수 있다.

- class를 선언하고 그 안에  __enter__와 __exit__ 메소드를 선언하면  그 class는 contextmethod로 작동한다.

In [6]:
# Ex3
# Use Class -> Context Manager with exception handling

class MyFileWriter():
    def __init__(self, file_name, method):
        print('MyFileWriter started : __init__')
        self.file_obj = open(file_name, method)
        
    def __enter__(self):
        print('MyFileWriter started : __enter__')
        return self.file_obj
    
    def __exit__(self, exc_type, value, trace_back):
        print('MyFileWriter started : __exit__')
        if exc_type:
            print('Logging exception:{}'.format(exc_type, value, trace_back))

        self.file_obj.close()
        
with MyFileWriter('./testfile3.txt','w') as f:
    f.write('Context Manager Test1 \n Contextlib Test3')
    

MyFileWriter started : __init__
MyFileWriter started : __enter__
MyFileWriter started : __exit__


context manager란?
프로그래밍 언어에서 데이터베이스 작업을 하거나 파일을 i/o(입,출력)하려나 외부 connection이 연결되는거는 
한정된 자원에서 하드웨어를 사용하는 것이기 때문에 서버의 운영 등 리소스가 제때 반환되지 않으면 시스템이 느려지거나 또는 
특정 상황에서 예외가 발생할 수 있다.
그럴때 자원의 수를 엄격한 타이밍에 하기 위해서 파이썬에서는 with문이 존재한다.
우리는 with문 동작 메커니즘을 통해서 enter, exit , class구현을 통해서 자원의 할당과 회수뿐만 아니라
우리가 원하는 logic을 추가하고 해당 패턴을 이용해서 우리가 원하는 class 형태를 만들어낼 수 있다.

In [9]:
# context manager를 통해서 내가 원하는 timer class를 만들 수 있다.

import time

class ExcuteTimer(object):
    def __init__(self, msg):
        self._msg = msg
        
    def __enter__(self):
        self._start = time.monotonic()
        return self._start
    
    def __exit__(self, exc_type, exc_value, exc_traceback):
        if exc_type:
            print("Logging exception {}".format((exc_type, exc_value, exc_traceback)))
        else:
            print("{} : {} s".format(self._msg, time.monotonic() - self._start)) 
        return True
    
with ExcuteTimer('Start! job') as v:
    print('Received start monotonic1 : {}'.format(v))
    # Excute job
    for i in range(1000000):
        pass
    
    raise Exception('Raise! Exception!') # 강제로 예외 발생

Received start monotonic1 : 996780.625
Logging exception (<class 'Exception'>, Exception('Raise! Exception!'), <traceback object at 0x0000027661FF6D00>)


In [10]:
# 가장 대표적인 with 구문 이해
# Contextlib 데코레이터 사용
# 코드 직관적, 예외 처리 용이성

import contextlib
import time

# Ex1 
# Use decorator

@contextlib.contextmanager
def my_file_writer(file_name, method):
    f = open(file_name, method)
    yield f # __enter__
    f.close() # __exit__

with my_file_writer('testfile4.txt', 'w') as f:
    f.write('Context Manager Test4.\nContextlib Test4')

In [13]:
# Use decorator

@contextlib.contextmanager
def ExcuteTimerDc(msg):
    start = time.monotonic()
    
    try: # __enter__
        yield start
    
    except BaseException as e:
        print('Logging exception:{}:{}'.format(msg, e))
        raise
    
    else: # __exit__
        print('{} : {} s'.format(msg, time.monotonic() - start))
        
with ExcuteTimerDc('Start job') as v:
    print('received start monotonic 2:{}'.format(v))
    
    # Execute job
    for i in range(101000):
        pass
    
    #raise ValueError('occurred')

received start monotonic 2:998293.656
Start job : 0.015000000013969839 s
